In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, confusion_matrix, precision_score, accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix

import pandas as pd

In [2]:
df = pd.read_csv("./input/clean_data/telecom_customer_churn.csv")

In [3]:
y = df["customerstatus"].values
X = df.drop(columns = "customerstatus")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 4, stratify =y)
col=['totalcharges','avgmonthlylongdistancecharges','monthlycharge','totalrevenue','totallongdistancecharges',
     'tenureinmonths','totallongdistancecharges','totalextradatacharges']

In [4]:
scaler = StandardScaler()
X_train[col] = StandardScaler().fit_transform(X_train[col])
X_test[col] = StandardScaler().fit_transform(X_test[col])

In [5]:
models = []
models.append(('Logistic Regression', LogisticRegression(max_iter=1000)))
models.append(('Kernel SVM', SVC()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('Gaussian NB', GaussianNB()))
models.append(('Random Forest', RandomForestClassifier()))
models.append(('Decision Tree Classifier', DecisionTreeClassifier()))

acc_results =[]
pre_results =[]
f1_results =[]
names = []

result_col = ["Algorithm", "Accuracy", "Precision", "f1 Score"]
model_results = pd.DataFrame(columns = result_col)

i=0
# K- fold cross validation

for name, model in models:
    names.append(name)
    # kfold = model_selection.KFold(n_splits=10)
    
    cv_acc_results = model_selection.cross_val_score(model, X_train, y_train, 
                     scoring="accuracy")
    cv_pre_results = model_selection.cross_val_score(model, X_train, y_train,
                     scoring="precision")
    cv_f1_results = model_selection.cross_val_score(model, X_train, y_train,
                     scoring="f1")
    acc_results.append(cv_acc_results)
    pre_results.append(cv_pre_results)
    f1_results.append(cv_f1_results)
    
    model_results.loc[i] = [name, 
                           round(cv_acc_results.mean()*100,2),
                           round(cv_pre_results.mean()*100,2),
                           round(cv_f1_results.mean()*100,2)]
    i+=1

model_results.sort_values(by = ['Accuracy'], ascending=False)

,Algorithm,Accuracy,Precision,f1 Score
4,Random Forest,86.56,88.31,91.05
0,Logistic Regression,84.39,89.37,89.06
5,Decision Tree Classifier,81.55,87.94,87.17
1,Kernel SVM,79.27,80.52,86.63
3,Gaussian NB,76.87,91.32,82.23
2,KNN,75.43,82.60,82.90


In [6]:
modelR = RandomForestClassifier()
modelR.fit(X_train, y_train)
pred_train = modelR.predict(X_train)
pred_test=modelR.predict(X_test)
acc_train = accuracy_score(pred_train, y_train)
acc_test = accuracy_score(pred_test, y_test)
print(f'Training accuracy {acc_train: .3f}') 
print(f'Testing accuracy {acc_test: .3f}')

Training accuracy  1.000
Testing accuracy  0.870


In [7]:
tuned_parameters = [{'n_estimators': [25, 50, 75],
                     'max_features': [15, 20, 25],
                     'max_leaf_nodes': [8, 16, 24]}] # hyperparameters to tune

clfR = GridSearchCV(RandomForestClassifier(), tuned_parameters, 
                   verbose=1, n_jobs=-1) # grid search model
clfR.fit(X_train, y_train) # evaluate hyper-parameters

print("\nBest parameters found:")
print(clfR.best_params_) # best hyperparameter values

print("\nGrid scores:")
means = clfR.cv_results_['mean_test_score'] # mean accuracy with folds
stds = clfR.cv_results_['std_test_score'] # standard deviation of accuracies
# for each hyperparameter combination show mean +/- 2 standard-deviations 
for mean, std, params in zip(means, stds, clfR.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" %(mean, std * 2, params)) 

Fitting 5 folds for each of 27 candidates, totalling 135 fits

Best parameters found:
{'max_features': 20, 'max_leaf_nodes': 24, 'n_estimators': 50}

Grid scores:
0.865 (+/-0.012) for {'max_features': 15, 'max_leaf_nodes': 8, 'n_estimators': 25}
0.869 (+/-0.015) for {'max_features': 15, 'max_leaf_nodes': 8, 'n_estimators': 50}
0.869 (+/-0.012) for {'max_features': 15, 'max_leaf_nodes': 8, 'n_estimators': 75}
0.866 (+/-0.008) for {'max_features': 15, 'max_leaf_nodes': 16, 'n_estimators': 25}
0.868 (+/-0.013) for {'max_features': 15, 'max_leaf_nodes': 16, 'n_estimators': 50}
0.870 (+/-0.015) for {'max_features': 15, 'max_leaf_nodes': 16, 'n_estimators': 75}
0.870 (+/-0.015) for {'max_features': 15, 'max_leaf_nodes': 24, 'n_estimators': 25}
0.868 (+/-0.010) for {'max_features': 15, 'max_leaf_nodes': 24, 'n_estimators': 50}
0.871 (+/-0.011) for {'max_features': 15, 'max_leaf_nodes': 24, 'n_estimators': 75}
0.866 (+/-0.014) for {'max_features': 20, 'max_leaf_nodes': 8, 'n_estimators': 25}
0

In [8]:
# Specify non-default hyper-parameter values
max_features = 15
max_leaf_nodes = 24
n_estimators = 75
# Train and test model
good_model = RandomForestClassifier(max_leaf_nodes = max_leaf_nodes,
                                    max_features = max_features, 
                                    n_estimators=n_estimators, ) # create model 
print(good_model) # display model parameters
good_model.fit(X_train, y_train) # train model
pred = good_model.predict(X_test) # predicted output for test examples
print("Results on test data")
acc = accuracy_score(y_test, pred) # accuracy on test examples
prec = precision_score(y_test, pred) # precision on test examples
reca = recall_score(y_test, pred) # recall on test examples
print(f'Test accuracy = {acc: .4f}') # round to 4 decimal places
print(f'Test precision = {prec: .4f}') # round to 4 decimal places
print(f'Test recall = {reca: .4f}') # round to 4 decimal places
print("Classification report:")
print(classification_report(y_test, pred))
print("Confusion matrix (Rows actual, Columns predicted):")
print(pd.DataFrame(confusion_matrix(y_test, pred)))

RandomForestClassifier(max_features=15, max_leaf_nodes=24, n_estimators=75)
Results on test data
Test accuracy =  0.8660
Test precision =  0.8720
Test recall =  0.9527
Classification report:
              precision    recall  f1-score   support

           0       0.84      0.65      0.73       561
           1       0.87      0.95      0.91      1416

    accuracy                           0.87      1977
   macro avg       0.86      0.80      0.82      1977
weighted avg       0.86      0.87      0.86      1977

Confusion matrix (Rows actual, Columns predicted):
     0     1
0  363   198
1   67  1349
